### Importing the libraries required

In [1]:
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor 

### Reading and checking the data

In [2]:
train=pd.read_csv('train.csv')
meal_info=pd.read_csv('meal_info.csv')
train.drop(columns=['id'], inplace=True)
train.head()

,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1,55,1885,136.83,152.29,0,0,177
1,1,55,1993,136.83,135.83,0,0,270
2,1,55,2539,134.86,135.86,0,0,189
3,1,55,2139,339.50,437.53,0,0,54
4,1,55,2631,243.50,242.50,0,0,40


In [3]:
train.set_index(['meal_id', 'center_id'],inplace=True)
train.head()

,,week,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
meal_id,center_id,,,,,,
1885,55,1,136.83,152.29,0,0,177
1993,55,1,136.83,135.83,0,0,270
2539,55,1,134.86,135.86,0,0,189
2139,55,1,339.50,437.53,0,0,54
2631,55,1,243.50,242.50,0,0,40


In [4]:
train.checkout_price.dtype

dtype('float64')

### Saving the models for all the splitted datasets 

In [5]:
for i in meal_info.meal_id.unique():
    for j in train.loc[i].index.unique():
        filename=f'{j}_{i}'
        df=pd.read_csv('SavedFilesBoth/'+filename+'.csv')
        df.drop(columns=['meal_id','center_id'], inplace=True)
        df.set_index(['week'],inplace=True)
        df[['checkout_price', 'base_price', 'diff']]/=100
        X= df.drop(columns=['num_orders']).values
        Y= df.num_orders.values
        X.reshape(-1,5)
        Y.reshape(-1)
        pipe1= Pipeline([('poly', PolynomialFeatures(degree=2, include_bias=True)),
                         ('ext', RandomForestRegressor( random_state=101, max_depth=8, min_impurity_decrease=0.5))])
        pipe1.fit(X, Y)
        df['y1']= pipe1.predict(X)
        X= df.drop(columns=['num_orders']).values
        Y= df.num_orders.values
        X.reshape(-1,6)
        Y.reshape(-1)
        pipe2= Pipeline([('poly', PolynomialFeatures(degree=6, include_bias=True)),
                         ('lin', BayesianRidge(tol= 1e-8, n_iter=300, normalize=True))])
        pipe2.fit(X, Y)
        joblib.dump(pipe1, (filename+'.sav'))
        joblib.dump(pipe2, (filename+'2.sav'))
